In [1]:
import PyNormaliz
from PyNormaliz import *
import numpy
from numpy import *
import itertools
from scipy.spatial import ConvexHull

In [2]:
import sys
sys.path.insert(0, '../Class/')
sys.path.insert(0,'../ClassAffine')
import integerSmithNormalFormAndApplications
from integerSmithNormalFormAndApplications import *
import AffineSemigroup
from AffineSemigroup import *
import auxiliars
from auxiliars import *

## Funciones auxiliares 

Veamos algunos ejemplos de semigrupos afines y veamos si son $\mathcal{C}$-semigrupos.

In [3]:
gen0 = [[3,0],[4,0],[5,0],[2,2],[3,3],[4,3],[5,3],[5,1],[6,1],[7,1],[5,4],[5,5]]

In [4]:
gen1 = [[3,3],[4,4],[5,5],[5,0],[5,1],[5,2],[5,3],[5,4],[6,0]]

In [5]:
ex10 = [[2,0,0],[4,2,4],[0,1,0],[3,0,0],[6,3,6],[3,1,1],[4,1,1],[3,1,2],[1,1,0],[3,2,3],[1,2,1]]

In [ ]:
# En primer lugar calculamos el cono de los generadores.

In [ ]:
C0 = Cone(cone=gen0)
C1 = Cone(cone=gen1)
C10 = Cone(cone=ex10)

In [ ]:
hp10=C10.SupportHyperplanes()

In [ ]:
rayos0 = C0.ExtremeRays()
rayos1 = C1.ExtremeRays()
rayos10 = C10.ExtremeRays()
rayos10

In [ ]:
q0 = len(rayos0)
q1 = len(rayos1)
q0,q1

Vemos ahora si en los ejes se forma un semigrupo.

In [6]:
# Check if all the coordinates are positives.
def BelongQPositive(v):
    for i in range(len(v)):
        if v[i] < 0:
            return False
    return True

In [7]:
# Pertenece a un eje y si es así, devuelve el múltiplo.
# INPUT:
#   - x: Value for checking if it is in the ray.
#   - r: Minimal value in the ray.
# OUTPUT:
#   - 0: If not belongs to the ray.
#   - A value if belongs to the ray.
def BelongAxis(x,r):
    coef = 0
    for i in range(len(x)):
        if x[i] != 0 and r[i] != 0:
            coef = x[i]/r[i]
            break
    if coef == 0:
        return 0
    aux2 = [j/coef for j in x]
    if aux2 == r:
        if(int(coef)==coef):
            return int(coef)
        return coef
    else:
        return 0

In [ ]:
BelongAxis([3,0],[1,0])

In [8]:
# Vemos para un eje si se forma un semigrupo.
# INPUT:
#   - gen: Set of generators.
#   - r: Minimal value in the ray.
# OUTPUT:
#   - True/False.
def AxisIsSemigroup(gen,r):
    aux = []
    for x in gen:
        aux.append(BelongAxis(x,r))
    aux2 = [x for x in aux if x != 0]
    if(gcdL(aux2) == 1):
        return True
    else:
        return False

In [ ]:
AxisIsSemigroup(ex10,rayos10[0]),AxisIsSemigroup(ex10,rayos10[1]),AxisIsSemigroup(ex10,rayos10[2])

In [9]:
# Vemos si los ejes forman un semigrupo.
# INPUT:
#   - gen: Set of generators.
#   - setR: Set of rays.
# OUTPUT:
#   - True/False.
def AxisAreSemigroup(gen,setR):
    aux = []
    for x in setR:
        aux.append(AxisIsSemigroup(gen,x))
    return all(aux)

In [ ]:
AxisAreSemigroup(gen0,rayos0)

In [ ]:
AxisAreSemigroup(gen1,rayos1)

In [ ]:
AxisAreSemigroup(ex10,rayos10)

Veamos ahora los puntos que hay dentro del *"diamantito"*

In [10]:
# En primer lugar tenemos que calcular el diamante.
# INPUT:
#   - a: Minimal elements of the ray.
# OUTPUT:
#   - Points of the diamond
def Diamond(a):
    aux = list(a)
    aux.append([0 for x in range(len(a))])
    for i in range(len(a)):
        for j in range(i+1,len(a)):
            aux.append(array(a[i])+array(a[j]))
    return [list(x) for x in  aux]

In [ ]:
diamante0 = Diamond(rayos0)
diamante1 = Diamond(rayos1)
diamante10 = Diamond(rayos10)
diamante10

In [ ]:
'''
from scipy.spatial import ConvexHull
points = np.random.rand(30, 2)
hull = ConvexHull(points)
import matplotlib.pyplot as plt
plt.plot(points[:,0], points[:,1], 'o')
for simplex in hull.simplices:
     plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
plt.plot(points[hull.vertices,0], points[hull.vertices,1], 'r--', lw=2)
plt.plot(points[hull.vertices[0],0], points[hull.vertices[0],1], 'ro')
plt.show()
'''

In [ ]:
hull0 = ConvexHull(diamante0)
eq0 = [list(x) for x in hull0.equations]
hull1 = ConvexHull(diamante1)
eq1 = [list(x) for x in hull1.equations]
hull10 = ConvexHull(diamante10)
eq10 = [list(x) for x in hull10.equations]
eq10

In [11]:
# Ahora veremos si un punto pertenece al diamante o no.
# INPUT:
#   - pt: Punto para comprobar la pertenencia.
#   - eq: Ecuaciones del diamante.
# OUTPUT:
#   - True/False.
def PointBelongsDiamond(pt,eq):
    dim = len(pt)
    for x in eq:
        sum = 0
        for i in range(dim):
            sum = sum + pt[i]*x[i]
        sum = round(sum+x[-1],10)
        if sum > 0:
            return False
    return True

In [ ]:
PointBelongsDiamond([1,1],eq0)

In [ ]:
PointBelongsDiamond([3,2,2],eq10)

In [12]:
# Calculo el menor cuboide que contenga al diamante.
# INPUT:
#   - d: Vértices del diamante.
# OUTPUT:
#   - Máximos en cada coordenada del cubo.
def Cube(d):
    dim = len(d[0])
    aux = []
    for i in range(dim):
        aux.append(sorted(d, key = lambda x: x[i])[-1][i])
    return(aux)

In [ ]:
cube0 = Cube(diamante0)
cube1 = Cube(diamante1)
cube10 = Cube(diamante10)
cube10

In [13]:
# Calculamos todos los puntos del diamante.
# INPUT:
#   - eq: Ecuaciones que definen el diamante.
# OUTPUT:
#   - Puntos enteros del diamante.
def IntegerDiamond(eq,cube):
    d = []
    it = itertools.product(*[range(i+1) for i in cube])
    for x in it:
        if PointBelongsDiamond(list(x),eq):
            d.append(list(x))
    return d

In [ ]:
diamanteEntero0 = IntegerDiamond(eq0,cube0)
diamanteEntero1 = IntegerDiamond(eq1,cube1)
diamanteEntero10 = IntegerDiamond(eq10,cube10)
diamanteEntero10

In [ ]:
hull11 = ConvexHull(diamante10+[cube10])
eq11 = [list(x) for x in hull11.equations]
diamanteEntero10 = IntegerDiamond(eq11,cube10)
diamanteEntero10

In [ ]:
eq=ConvexHull(Diamond([[1,3],[5,2]])).equations
IntegerDiamond(eq,[6,5])

A continuación, veremos para cada rayo el semigrupo afín generado por los términos independientes de las ecuaciones en los puntos enteros del diamante.

In [ ]:
'''C0.SupportHyperplanes()'''

In [ ]:
'''Cone(cone=[[1,0,0],[3,4,5],[4,5,5]]).SupportHyperplanes()'''

In [14]:
def ProdEsc(v1,v2):
    n = len(v1)
    suma = 0
    for i in range(n):
        suma = suma + v1[i] * v2[i]
    return suma

In [ ]:
ProdEsc([1,2,3],[1,1,1])

In [15]:
# En primer lugar calculamos las ecuaciones que definen un rayo.
# INPUT:
#   - ray: rayo del cono.
#   - hp: hiperplanos soportes.
# OUTPUT:
#   - ecuaciones del rayo.
def EqRay(ray,hp):
    eq = []
    for x in hp:
        if ProdEsc(ray,x) == 0:
            eq.append(x)
    return eq

In [ ]:
EqRay([2,1,2],hp10)

In [ ]:
EqRay(rayos10[0],hp10),EqRay(rayos10[1],hp10),EqRay(rayos10[2],hp10)

In [16]:
# Eliminamos los elementos que son todos ceros.
# INPUT:
#   - m: matriz.
# OUTPUT:
#   - Matriz m sin filas nulas.
def DeleteRowZero(m):
    aux = []
    for v in m:
        allzero = True
        for i in range(len(v)):
            if v[i] != 0:
                allzero = False
        if not allzero:
            aux.append(v)
    return aux

In [ ]:
DeleteRowZero([[0,1],[0,0],[1,0],[1]])

In [17]:
# Borramos elementos repetidos.
# INPUT:
#   - v: vector de entrada.
# OUTPUT:
#   - vector v sin elementos de entrada.
def DeleteDuplicates(v):
    w = []
    for i in range(len(v)):
        if v[i] not in w:
            w.append(v[i])
    return w

In [18]:
# Calculamos los valores afines del diamante con respecto a un rayo.
# INPUT:
#   - eq: Ecuaciones del rayo.
#   - d: Diamante entero.
# OUTPUT:
#   - Terminos afines del diamante sin los ceros ni elementos duplicados.
def AffineTerm(eq,d):
    neq = len(eq)
    dim = len(eq[0])
    afin = []
    for x in d:
        aux = []
        for e in eq:
            aux = aux + [ProdEsc(x,e)]
        afin.append(aux)
    return DeleteDuplicates(DeleteRowZero(afin))

In [ ]:
e0 = EqRay(rayos10[0],hp10)
e1 = EqRay(rayos10[1],hp10)
e2 = EqRay(rayos10[2],hp10)

In [ ]:
AffineTerm(e0,diamanteEntero10)

In [ ]:
AffineTerm(e1,diamanteEntero10)

In [ ]:
AffineTerm(e2,diamanteEntero10)

In [ ]:
afs=AffineSemigroup(AffineTerm([[0,1]],diamanteEntero0), "generators")
print(afs.getMSG())

In [19]:
# Multiplicar matrices.
# INPUT:
#   - X: una matriz.
#   - Y: otra matriz.
# OUTPUT:
#   - Producto de ambas.
def MultiplyMatrix(X,Y):
    result = [[0 for y in Y[0]] for x in X]
    # iterate through rows of X
    for i in range(len(X)):
        # iterate through columns of Y
        for j in range(len(Y[0])):
            # iterate through rows of Y
            for k in range(len(Y)):
                result[i][j] += X[i][k] * Y[k][j]
    return result

In [ ]:
MultiplyMatrix([[12,7,3],[4 ,5,6],[7 ,8,9]],[[5],[6],[4]])

In [ ]:
MultiplyMatrix([[0,1]],[[5],[1]])

In [60]:
# Calculamos si en cada recta afín paralela al rayo hay un generador.
# INPUT:
#   - eqray: ecuaciones de un rayo.
#   - afinset: valor afín de la recta.
#   - smg: generadores del semigrupo.
# OUTPUT:
#   True/False si en esa recta hay un generador.
def ExistGenerator(eqray, afin,smg):
    aux = AffineTerm(eqray,smg)
    if afin in aux:
        return True
    else:
        return False

In [61]:
ExistGenerator([[0, 0, 1], [1, 0, -1]],[0, 1],ex10)

True

## Función algoritmo 1

In [62]:
gen0

[[3, 0], [4, 0], [5, 0], [2, 2], [3, 3], [4, 3], [5, 3], [5, 1], [6, 1], [7, 1
], [5, 4], [5, 5]]

In [63]:
# Esta función dice si un conjunto de elementos genera o no un C-semigrupo.
# INPUT:
#   - smg: Sistema generador.
# OUTPUT:
#   - True/False.
def IsCsemigroup(smg):
    # En primer lugar calculamos los rayos del cono con Pynormaliz.
    cono = Cone(cone=smg)
    rayos = cono.ExtremeRays()
    # Calculamos también los hiperplanos soportes.
    hp = cono.SupportHyperplanes()
    for ray in rayos:
        # En primer lugar comprobamos que el rayo es un semigrupo en sí mismo.
        if not AxisIsSemigroup(smg,ray):
            return False
    # Calculamos el diamante.
    diamante = Diamond(rayos)
    # Calculamos sus ecuaciones gracias a scipy.spatial -> ConvexHull.
    hull = ConvexHull(diamante)
    eqDiamante = [list(x) for x in hull.equations]
    # Calculamos una cota para los puntos del diamante.
    cotaDiamante = Cube(diamante)
    # Calculamos el diamante entero.
    diamanteEntero = IntegerDiamond(eqDiamante,cotaDiamante)
    # Veamos ahora que por cada rayo las paralelas afines "generadoras" cortan a los generadores del semigrupo.
    for ray in rayos:
        # Calculamos las ecuaciones del rayo.
        eqrayo = EqRay(ray,hp)
        # Calculamos los términos afines de los puntos enteros del diamantes.
        afinesDiamante = AffineTerm(eqrayo,diamanteEntero)
        # Nos quedamos con los generadores afines.
        genAfines = AffineSemigroup(afinesDiamante, "generators").getMSG()
        # Comprobamos que por cada afín pasa un generador
        for afin in genAfines:
            if not ExistGenerator(eqrayo, afin,smg):
                return False
    return True

In [64]:
IsCsemigroup(ex10)

True

In [27]:
IsCsemigroup(gen0)

El rayo es:  [1, 0]
El rayo es:  [1, 1]


True

In [23]:
gen0

[[3, 0], [4, 0], [5, 0], [2, 2], [3, 3], [4, 3], [5, 3], [5, 1], [6, 1], [7, 1
], [5, 4], [5, 5]]

In [24]:
IsCsemigroup(gen1)

True

## Funciones auxiliares 2 

In [ ]:
# En primer lugar tenemos que calcular los conductores en cada rayo.

In [ ]:
rayos0

In [ ]:
BelongAxis(gen0[0],rayos0[0])

In [ ]:
# Calculamos el conductor.
# INPUT:
#   - gen: Set of generators.
#   - r: Minimal value in the ray.
# OUTPUT:
#   - conductor
def ConductorAxis(gen,r):
    aux = []
    for x in gen:
        aux.append(BelongAxis(x,r))
    aux2 = [x for x in aux if x != 0]
    print(r,aux2)
    return [x*(FrobeniusNumber(aux2)+1) for x in r]
    

In [ ]:
ConductorAxis(gen0,rayos0[1])